#### 1. Importing Libraries

In [1]:
import sys
import requests
import bs4
import json
import re
import time
from nltk.tokenize import sent_tokenize
import re

#### 2. Extracting WikiPages for all Hindi Labelled Monuments

In [2]:
monument_list = []
for i in range(1,120):
    with open('../Data/Wikidata_JSON/en_monument_dump_part'+str(i)+'.json') as f:
        new_list = json.load(f)
        monument_list = monument_list + new_list

In [3]:
t0 = time.time()
articles = {}
count_monuments = 1600
for monument in monument_list[1600:]:
    res = requests.get('https://en.wikipedia.org/wiki/' + monument['labels']['en']['value'])
    if res.status_code != 404:
        wiki = bs4.BeautifulSoup(res.text,"lxml")
        elems = wiki.select('p')
        article = []
        for i in range(len(elems)):
            text = elems[i].getText().replace('\n', '').lower()
            text = text.replace('\ufeff', ' ')
            text = re.sub('\[\d+\]', '', text.strip())
            article.extend(sent_tokenize(text))
    
        if '' in article:
            article.remove('')
        
        for i in range(0,len(article)):
            article[i] = article[i].strip()
            
        articles[monument['id']] = article
    
    count_monuments = count_monuments + 1
    
    if count_monuments%200 == 0:
        partition_num = str(count_monuments/200).split('.')[0]
        with open('../Data/English_Wikipages/en_articles_part'+ partition_num + '.json', 'w') as fout:
            json.dump(articles, fout)
        
        articles = {}
        
        print("Checkpoint %d reached, JSON dumps saved |" % (count_monuments/200), end = ' ')
        print("Time Elapsed:", end = ' ')
        print(time.time()-t0)

t1 = time.time()
total = t1-t0

Checkpoint 9 reached, JSON dumps saved | Time Elapsed: 358.67494773864746
Checkpoint 10 reached, JSON dumps saved | Time Elapsed: 734.5727803707123
Checkpoint 11 reached, JSON dumps saved | Time Elapsed: 1088.4423913955688
Checkpoint 12 reached, JSON dumps saved | Time Elapsed: 1452.0951673984528
Checkpoint 13 reached, JSON dumps saved | Time Elapsed: 1838.1248536109924
Checkpoint 14 reached, JSON dumps saved | Time Elapsed: 2206.3132951259613
Checkpoint 15 reached, JSON dumps saved | Time Elapsed: 2573.7488400936127
Checkpoint 16 reached, JSON dumps saved | Time Elapsed: 2942.321704387665
Checkpoint 17 reached, JSON dumps saved | Time Elapsed: 3299.2757494449615
Checkpoint 18 reached, JSON dumps saved | Time Elapsed: 3675.6000261306763
Checkpoint 19 reached, JSON dumps saved | Time Elapsed: 4022.9200007915497
Checkpoint 20 reached, JSON dumps saved | Time Elapsed: 4389.505580663681
Checkpoint 21 reached, JSON dumps saved | Time Elapsed: 4771.039614677429
Checkpoint 22 reached, JSON du

In [4]:
print(total)
print(len(articles))

18397.479632139206
11


In [5]:
tot_len = 0
for i in range(1,57):
    with open('../Data/English_Wikipages/en_articles_part'+str(i)+'.json') as f:
        tot_len += len(json.load(f))
tot_len

2011